What the data looks like 

for each episode there is a json-file with following structure

{"results":[{"alternatives":[{"transcript":"Hi, this is transcript of first segment","confidence":0.91,"words":[{
                     "startTime":"0s",
                     "endTime":"0.300s",
                     "word":"Hi,"
                  },
                  {"startTime":"0.500s",
                     "endTime":"0.900s",
                     "word":"this"}]}]}]}




What I want the data to look like (kind of)

for each show there shoud be one dict containing show id, show name and all episodes of show split into segments

[{'show-id':'123','show-name':'alex & sigge','episodes':[{'name':'first episode','date':'2020-01-01','segmentTranscripts':[{'segNo':0,'startTime':'00:00s','endTime':'00:20s','transcript':'......',},{},]}]},{},]

- show name would have to be looked up in the metadata.tsv

In [4]:
import json

#for each episode extract {word:'hello',timestamp:'00.00s'} 
    

def extract_segments(path):
    with open(path, "r") as read_file:
        episode = json.load(read_file)
    segments=[]
    #had to do "manual" iteration due to irregularities in data
    iter=0
    for segment in episode["results"]:
        seg_result={}
        #make sure there is only one dict in this list (should be true according to dataset description)
        assert len(segment["alternatives"])==1
        segment_dict=segment["alternatives"][0]
        #sometimes "alternatives" dict is empty...
        if "words" and "transcript"  in segment_dict:
            #add segment number
            seg_result["segNum"]=iter
            #add timestamp of the first word in this segment
            seg_result["startTime"]=segment_dict["words"][0]["startTime"]
            #add timestamp of the last word in this segment
            seg_result["endTime"]=segment_dict["words"][-1]["endTime"]
            #add transcript of this segment 
            seg_result["transcript"]=segment_dict["transcript"]
            segments.append(seg_result)
            iter+=1

    return segments


In [17]:
extract_segments('/Users/Simpan/spotify/spotify-podcasts-2020/podcasts-transcripts/7/0/show_70AtBgIej68YuFXDt6l0aB/3GsS8PAYL6u3ClsZFnW3RZ.json').keys()

dict_keys(['3GsS8PAYL6u3ClsZFnW3RZ'])

In [7]:
from training_data_collection import collect_training_episodes

input_file = '/Users/Simpan/spotify/Podcast-Segment-Retrieval-Spotify/data/podcasts_2020_train.1-8.qrels.txt'
root_dir = '/Users/Simpan/spotify/Podcast-Segment-Retrieval-Spotify/data/podcasts-no-audio-13GB'
#training_episodes = collect_training_episodes(root_dir, input_file)
training_segments = {}
training_episodes=['/Users/Simpan/spotify/spotify-podcasts-2020/podcasts-transcripts/6/0/show_60aEckwTYs8xCEpsAasV0o/3NHTGeZoLLIfoHnlwtOu6w.json']
for episode in training_episodes:
    episode_id=episode.split('/')[-1].split('.json')[0]
    training_segments[episode_id]=extract_segments(episode)
with open('../data/training_sub_test.json', 'w') as fout:
        json.dump(training_segments , fout)

{'3NHTGeZoLLIfoHnlwtOu6w': [{'segNum': 0, 'startTime': '0s', 'endTime': '14.900s', 'transcript': "Hi, I'm Kenya and I am one year in you're listening to the GC youth Ministries podcast. You plus me plus we welcome welcome pull up a chair grab a seed and listening."}, {'segNum': 1, 'startTime': '29.800s', 'endTime': '57.300s', 'transcript': " All right, so welcome. I'm Anya. Hi Kenya. Are you feeling tired tired? You don't know dying. Hmm. I can assure you that much. I can only imagine so hello to everyone that's listening in welcome. Hey, how are you sir? How are y'all? How are you doing? Kenya? You asked me how I was doing. Yeah. Look at that politeness."}, {'segNum': 2, 'startTime': '58.400s', 'endTime': '87.300s', 'transcript': " Not that person. I'm doing good doing better feeling excited. I'm excited about this. Me too. Yeah, because I feel like this is fine like our baby. Hmm. I think Kenny and I have been trying to do a podcast for like a year outside of thousands of years and t

In [4]:
import json
path='../data/training_sub.json'
with open(path,'r') as f:
    data=json.load(f)
data[0][0]

{'segNum': 0,
 'startTime': '14.100s',
 'endTime': '21.400s',
 'transcript': "Hello, this is Zach. You're welcome to st. Patrick's podcast. This is presented to you by us Zack Caden."}

In [ ]:
#fetch queries from xml and put into list
#def fetch_queries(path)
import xml.etree.ElementTree as ET
tree = ET.parse('../../podcasts_2020_topics_train.xml')
root = tree.getroot()

topics=[{'topic-no':element[0].text,'query':element[2].text,'description':element[3].text} for element in root]


In [34]:
#fetch topic number, correct jump-in point for training set

with open('../../podcasts_2020_train.1-8.qrels.txt','r') as f:
    contents=f.readlines()
targets=[{'episode':line.split()[2].split('_')[0].split(':')[2],'topic-no':line[0],'target-time':line.split()[2].split('_')[1]} for line in contents]

len(targets)

609

In [ ]:
#TODO 
#match episodes, target times and queries
#rewrite semi pseudo-code
from sentence_transformers import SentenceTransformer, util
import numpy as np


embedder = SentenceTransformer('bert-base-nli-mean-tokens')


#semi pseduo-code
n_correct=0
for episode in data:
    episode_id=
    target_time=
    data=extract_segments(path)
    segments=[item["transcript"] for item in data]
    timespans=[(item["startTime"],item["stopTime"]) for item in data]
    query=topics[episode['episode-no']]['query']
    episode_embeddings=embedder.encode(segments, convert_to_tensor=True)
    query_embedding=embedder(query)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    #if running on gpu
    #if cuda.device()==GPU
    #cos_scores = cos_scores.cpu()
    top_k=5
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    idx_closest=top_results[0]
    pred_timespan=timespans[idx_closest]
    if pred_timespan[0]<=target_timespan<=pred_timespan[1]:
        n_correct+=1


accuracy=n_correct/len(test_dataset)





In [ ]:
segments_l

In [ ]:
#dont run
import os

base_dir="/Users/Simpan/spotify/spotify-podcasts-2020/podcasts-transcripts"
data=[]
for sub_dir in os.listdir(base_dir):
    for sub_sub_dir in os.listdir(base_dir+'/'+sub_dir):
        for show_dir in os.listdir(base_dir+'/'+sub_dir+'/'+sub_sub_dir):
            show_dict={}
            show_dict["showId"]=show_dir
            show_dict["episodes"]=[]
            for i,episode in enumerate(os.listdir(base_dir+'/'+sub_dir+'/'+sub_sub_dir+'/'+show_dir)):
                episode_dict={}
                #remove .json ending
                episode_dict["episodeId"]=episode[:-5]
                episode_dict["episodeNo"]=i
                episode_path=base_dir+'/'+sub_dir+'/'+sub_sub_dir+'/'+show_dir+'/'+episode
                episode_dict["episodeSegments"]=extract_segments(episode_path)
                show_dict["episodes"].append(episode_dict)
            data.append(show_dict)
with open("test.json", "w+") as write_file:
    json.dump(data, write_file)